In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

c:\Users\V Nikhil\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the Bayesian Network structure
model = BayesianNetwork([
    ('T', 'S'),  # Theft causes Siren
    ('Q', 'S'),  # Quake causes Siren
    ('S', 'K'),  # Siren causes Kevin to call
    ('S', 'L')   # Siren causes Lisa to call
])

In [3]:
# Define the CPDs with new values
cpd_t = TabularCPD('T', 2, [[0.995], [0.005]])
cpd_q = TabularCPD('Q', 2, [[0.997], [0.003]])
cpd_s = TabularCPD('S', 2,
                   [[0.998, 0.8, 0.5, 0.1],
                    [0.002, 0.2, 0.5, 0.9]],
                   evidence=['T', 'Q'], evidence_card=[2, 2])
cpd_k = TabularCPD('K', 2,
                   [[0.9, 0.2],
                    [0.1, 0.8]],
                   evidence=['S'], evidence_card=[2])
cpd_l = TabularCPD('L', 2,
                   [[0.85, 0.3],
                    [0.15, 0.7]],
                   evidence=['S'], evidence_card=[2])

In [4]:
# Add the CPDs to the model
model.add_cpds(cpd_t, cpd_q, cpd_s, cpd_k, cpd_l)

# Validate the model
assert model.check_model()

In [5]:
# Print the full CPTs
for cpd in model.get_cpds():
    print(f"\nCPD of {cpd.variable}:")
    print(cpd)

# Perform inference
inference = VariableElimination(model)

# Query: P(T | K=true, L=true)
result = inference.query(variables=['T'], evidence={'K': 1, 'L': 1})
print("\nP(T | K=true, L=true):")
print(result)


CPD of T:
+------+-------+
| T(0) | 0.995 |
+------+-------+
| T(1) | 0.005 |
+------+-------+

CPD of Q:
+------+-------+
| Q(0) | 0.997 |
+------+-------+
| Q(1) | 0.003 |
+------+-------+

CPD of S:
+------+-------+------+------+------+
| T    | T(0)  | T(0) | T(1) | T(1) |
+------+-------+------+------+------+
| Q    | Q(0)  | Q(1) | Q(0) | Q(1) |
+------+-------+------+------+------+
| S(0) | 0.998 | 0.8  | 0.5  | 0.1  |
+------+-------+------+------+------+
| S(1) | 0.002 | 0.2  | 0.5  | 0.9  |
+------+-------+------+------+------+

CPD of K:
+------+------+------+
| S    | S(0) | S(1) |
+------+------+------+
| K(0) | 0.9  | 0.2  |
+------+------+------+
| K(1) | 0.1  | 0.8  |
+------+------+------+

CPD of L:
+------+------+------+
| S    | S(0) | S(1) |
+------+------+------+
| L(0) | 0.85 | 0.3  |
+------+------+------+
| L(1) | 0.15 | 0.7  |
+------+------+------+

P(T | K=true, L=true):
+------+----------+
| T    |   phi(T) |
+======+==========+
| T(0) |   0.9189 |
+------